In [1]:
# ============================================
# 1. INSTALL DEPENDENCIES
# ============================================

print("Installing dependencies...")
!pip install -q -U pip
!pip install -q -U "transformers>=4.31.0" "huggingface_hub>=0.18.0"
!pip install -q git+https://github.com/csebuetnlp/normalizer
!pip install -q evaluate seqeval datasets accelerate


Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'normalizer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'normalizer'. D

In [2]:
# ============================================
# 2. IMPORTS
# ============================================

print("\nImporting libraries...")
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import evaluate
import numpy as np



Importing libraries...


2025-10-23 11:18:38.570182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761218318.993539      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761218319.107323      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ============================================
# 3. GPU CHECK
# ============================================

print("\n" + "="*60)
print("GPU DIAGNOSTIC CHECK")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device("cuda")
    print("✓ GPU is available and will be used!")
else:
    device = torch.device("cpu")
    print("⚠️ WARNING: No GPU detected! Training will be SLOW on CPU")
    print("In Colab: Runtime > Change runtime type > Hardware accelerator > GPU")



GPU DIAGNOSTIC CHECK
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU count: 2
GPU name: Tesla T4
GPU memory: 15.83 GB
✓ GPU is available and will be used!


In [4]:
# ============================================
# 4. LOAD DATASETS
# ============================================

print("\n" + "="*60)
print("LOADING WIKIANN TELUGU DATASET (Training)")
print("="*60)
train_dataset = load_dataset("wikiann", "te")
print("Telugu dataset for training:")
print(train_dataset)
print(f"\nTrain size: {len(train_dataset['train'])}")
print(f"Validation size: {len(train_dataset['validation'])}")

print("\n" + "="*60)
print("LOADING WIKIANN BANGLA DATASET (Testing - Cross-lingual Transfer)")
print("="*60)
test_dataset = load_dataset("wikiann", "bn")
print("Bangla dataset for testing:")
print(test_dataset)
print(f"Test size: {len(test_dataset['test'])}")

# Use Telugu for training/validation, Bangla for test
dataset = {
    "train": train_dataset["train"],
    "validation": train_dataset["validation"],
    "test": test_dataset["test"]
}

print("\nSample from Telugu training set:")
print(dataset["train"][0])

label_list = dataset["train"].features["ner_tags"].feature.names
print(f"\nNER Labels: {label_list}")

print("\n" + "-"*60)
print("SAMPLE TELUGU SENTENCES WITH TAGS (Training Data)")
print("-"*60)
for i in range(3):
    tokens = dataset["train"][i]["tokens"]
    tags = [label_list[t] for t in dataset["train"][i]["ner_tags"]]
    print(f"\nSentence {i+1}:")
    print("Tokens:", " ".join(tokens))
    print("Tags:  ", " ".join(tags))

print("\n" + "-"*60)
print("SAMPLE BANGLA SENTENCES (Test Data - Cross-lingual Evaluation)")
print("-"*60)
for i in range(2):
    tokens = dataset["test"][i]["tokens"]
    tags = [label_list[t] for t in dataset["test"][i]["ner_tags"]]
    print(f"\nSentence {i+1}:")
    print("Tokens:", " ".join(tokens))
    print("Tags:  ", " ".join(tags))


LOADING WIKIANN TELUGU DATASET (Training)


README.md: 0.00B [00:00, ?B/s]

te/validation-00000-of-00001.parquet:   0%|          | 0.00/86.1k [00:00<?, ?B/s]

te/test-00000-of-00001.parquet:   0%|          | 0.00/86.8k [00:00<?, ?B/s]

te/train-00000-of-00001.parquet:   0%|          | 0.00/87.9k [00:00<?, ?B/s]

...

bn/train-00000-of-00001.parquet:   0%|          | 0.00/554k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Bangla dataset for testing:
DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
})
Test size: 1000

Sample from Telugu training set:
{'tokens': ['ఆంధ్ర', 'ప్రదేశ్లో', ',', 'వారు', 'జనాభాలో', '2', '%', 'కంటే', 'తక్కువగాను', ',', 'తమిళనాడులో', 'వారు', '3', '%', 'కన్నా', 'తక్కువ', 'మంది', 'ఉన్నారు', '.'], 'ner_tags': [5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0], 'langs': ['te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te'], 'spans': ['LOC: ఆంధ్ర ప్రదేశ్లో', 'LOC: తమిళనాడులో']}

NER Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

------------------------------------------------------------
SAMPLE TELU

In [5]:
# ============================================
# 5. LOAD TOKENIZER — INDICBERT
# ============================================

print("\n" + "="*60)
print("LOADING TOKENIZER (INDICBERT)")
print("="*60)
MODEL_NAME = "ai4bharat/IndicBERTv2-MLM-only"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer loaded: {MODEL_NAME}")



LOADING TOKENIZER (INDICBERT)


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Tokenizer loaded: ai4bharat/IndicBERTv2-MLM-only


In [6]:
# ============================================
# 6. TOKENIZATION FUNCTION
# ============================================

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False,
        max_length=128
    )

    all_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


In [7]:
# ============================================
# 7. TOKENIZE DATASET
# ============================================

print("\n" + "="*60)
print("TOKENIZING DATASETS")
print("="*60)

# Tokenize each split separately
tokenized_train = dataset["train"].map(
    tokenize_and_align_labels,
    batched=True,
    desc="Tokenizing Telugu train"
)
tokenized_val = dataset["validation"].map(
    tokenize_and_align_labels,
    batched=True,
    desc="Tokenizing Telugu validation"
)
tokenized_test = dataset["test"].map(
    tokenize_and_align_labels,
    batched=True,
    desc="Tokenizing Bangla test"
)

# Remove unnecessary columns
tokenized_train = tokenized_train.remove_columns(["tokens", "ner_tags", "langs"])
tokenized_val = tokenized_val.remove_columns(["tokens", "ner_tags", "langs"])
tokenized_test = tokenized_test.remove_columns(["tokens", "ner_tags", "langs"])

# Create the tokenized_datasets dictionary
tokenized_datasets = {
    "train": tokenized_train,
    "validation": tokenized_val,
    "test": tokenized_test
}

print("Tokenized datasets:")
print(f"  Train: {len(tokenized_datasets['train'])} samples")
print(f"  Validation: {len(tokenized_datasets['validation'])} samples")
print(f"  Test (Bangla): {len(tokenized_datasets['test'])} samples")


TOKENIZING DATASETS


Tokenizing Telugu train:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing Telugu validation:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing Bangla test:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenized datasets:
  Train: 1000 samples
  Validation: 1000 samples
  Test (Bangla): 1000 samples


In [8]:
# ============================================
# 8. LOAD MODEL — INDICBERT
# ============================================

print("\n" + "="*60)
print("LOADING MODEL (INDICBERT)")
print("="*60)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_list),
    id2label={i: label for i, label in enumerate(label_list)},
    label2id={label: i for i, label in enumerate(label_list)}
)
model = model.to(device)
print(f"Model loaded: {MODEL_NAME}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Number of parameters: {model.num_parameters():,}")



LOADING MODEL (INDICBERT)


config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: ai4bharat/IndicBERTv2-MLM-only
Model device: cuda:0
Number of parameters: 277,456,135


In [9]:
# ============================================
# 9. SETUP METRICS
# ============================================

print("\n" + "="*60)
print("LOADING METRICS")
print("="*60)
metric = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



LOADING METRICS


In [10]:
# ============================================
# 10. DATA COLLATOR
# ============================================

print("\n" + "="*60)
print("SETTING UP DATA COLLATOR")
print("="*60)
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100
)
print("Data collator created successfully!")



SETTING UP DATA COLLATOR
Data collator created successfully!


In [11]:
# ============================================
# 11. TRAINING ARGUMENTS
# ============================================

print("\n" + "="*60)
print("SETTING UP TRAINING ARGUMENTS")
print("="*60)

training_args = TrainingArguments(
    output_dir="./results-indicbert-telugu-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    save_total_limit=2,
    report_to="none",
)

print(f"Training device: {training_args.device}")
print(f"FP16 training: {training_args.fp16}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Number of epochs: {training_args.num_train_epochs}")
print("\n⚠️ NOTE: Training on TELUGU data, Testing on BANGLA data (Cross-lingual Transfer)")


SETTING UP TRAINING ARGUMENTS
Training device: cuda:0
FP16 training: True
Batch size: 16
Number of epochs: 5

⚠️ NOTE: Training on TELUGU data, Testing on BANGLA data (Cross-lingual Transfer)


In [12]:
# ============================================
# 12. CREATE TRAINER
# ============================================

print("\n" + "="*60)
print("CREATING TRAINER")
print("="*60)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Trainer created successfully!")



CREATING TRAINER
Trainer created successfully!


In [13]:
# ============================================
# 13. START TRAINING
# ============================================

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"This will take approximately 15–25 minutes on a T4 GPU")
print("="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("TRAINING COMPLETED!")
print("="*60)


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 3}.



STARTING TRAINING
This will take approximately 15–25 minutes on a T4 GPU



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.756556,0.000000,0.000000,0.000000,0.719183
2,1.009000,0.562065,0.411950,0.218152,0.285248,0.799381
3,1.009000,0.382012,0.578065,0.631973,0.603819,0.885396
4,0.499600,0.309965,0.624714,0.681932,0.652070,0.909282
5,0.295300,0.291564,0.657729,0.694421,0.675577,0.914233


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/di


TRAINING COMPLETED!


In [14]:
# ============================================
# 14. FINAL EVALUATION
# ============================================

print("\n" + "="*60)
print("FINAL EVALUATION ON TELUGU VALIDATION SET")
print("="*60)
eval_results = trainer.evaluate()
print("\nTelugu Validation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

print("\n" + "="*60)
print("CROSS-LINGUAL EVALUATION ON BANGLA TEST SET")
print("="*60)
print("⚠️ Testing cross-lingual transfer: Telugu-trained model on Bangla data")
test_results = trainer.evaluate(tokenized_datasets["test"])
print("\nBangla Test Results (Cross-lingual Transfer):")
for key, value in test_results.items():
    print(f"  {key}: {value:.4f}")

print("\n" + "="*60)
print("TRANSFER LEARNING ANALYSIS")
print("="*60)
print(f"Telugu → Telugu F1: {eval_results['eval_f1']:.4f}")
print(f"Telugu → Bangla F1: {test_results['eval_f1']:.4f}")
transfer_gap = eval_results['eval_f1'] - test_results['eval_f1']
print(f"Transfer Gap: {transfer_gap:.4f}")
if transfer_gap > 0:
    print("✓ Model performs better on source language (expected)")
else:
    print("⚠️ Model performs better on target language (unexpected)")


FINAL EVALUATION ON TELUGU VALIDATION SET


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Telugu Validation Results:
  eval_loss: 0.2916
  eval_precision: 0.6577
  eval_recall: 0.6944
  eval_f1: 0.6756
  eval_accuracy: 0.9142
  eval_runtime: 4.1146
  eval_samples_per_second: 243.0370
  eval_steps_per_second: 7.7770
  epoch: 5.0000

CROSS-LINGUAL EVALUATION ON BANGLA TEST SET
⚠️ Testing cross-lingual transfer: Telugu-trained model on Bangla data

Bangla Test Results (Cross-lingual Transfer):
  eval_loss: 1.0849
  eval_precision: 0.3289
  eval_recall: 0.4536
  eval_f1: 0.3813
  eval_accuracy: 0.6541
  eval_runtime: 4.0046
  eval_samples_per_second: 249.7100
  eval_steps_per_second: 7.9910
  epoch: 5.0000

TRANSFER LEARNING ANALYSIS
Telugu → Telugu F1: 0.6756
Telugu → Bangla F1: 0.3813
Transfer Gap: 0.2943
✓ Model performs better on source language (expected)


In [15]:
# ============================================
# 15. SAVE MODEL
# ============================================

print("\n" + "="*60)
print("SAVING FINAL MODEL")
print("="*60)
trainer.save_model("./indicbert-telugu-ner-final")
tokenizer.save_pretrained("./indicbert-telugu-ner-final")
print("Model saved to: ./indicbert-telugu-ner-final")
print("This model was trained on Telugu and can be used for cross-lingual transfer to Bangla")


SAVING FINAL MODEL
Model saved to: ./indicbert-telugu-ner-final
This model was trained on Telugu and can be used for cross-lingual transfer to Bangla


In [16]:
import shutil
import os

# Define the output zip file
output_filename = "kaggle_working_dir.zip"

# Current working directory
current_dir = os.getcwd()

# Create a zip of the current directory
shutil.make_archive("kaggle_working_dir", 'zip', current_dir)

print(f"✅ ZIP file created: {output_filename}")


✅ ZIP file created: kaggle_working_dir.zip


In [17]:
# ============================================
# 16. TEST INFERENCE
# ============================================

print("\n" + "="*60)
print("TESTING INFERENCE")
print("="*60)

test_sentence = dataset["test"][0]
test_tokens = test_sentence["tokens"]
print(f"\nTest sentence: {' '.join(test_tokens)}")

inputs = tokenizer(
    test_tokens,
    is_split_into_words=True,
    return_tensors="pt",
    truncation=True,
    padding=True
).to(device)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

predicted_labels = [label_list[p.item()] for p in predictions[0]]
word_ids = inputs.word_ids()

final_predictions = []
previous_word_idx = None
for word_idx, pred_label in zip(word_ids, predicted_labels):
    if word_idx is not None and word_idx != previous_word_idx:
        final_predictions.append(pred_label)
        previous_word_idx = word_idx

print("\nPredicted NER tags:")
for token, pred_tag in zip(test_tokens, final_predictions):
    print(f"  {token:20s} -> {pred_tag}")

print("\n" + "="*60)
print("ALL DONE! 🎉")
print("="*60)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



TESTING INFERENCE

Test sentence: উরুগুয়ে জাতীয় ফুটবল দল

Predicted NER tags:
  উরুগুয়ে             -> B-LOC
  জাতীয়               -> I-ORG
  ফুটবল                -> I-ORG
  দল                   -> I-ORG

ALL DONE! 🎉
